In [1]:
import requests
import json
import pandas as pd
import re
from pprint import pprint

# Import the API key
from config import ombdapi_key

In [2]:
# read oscar winning movie list
oscar_df = pd.DataFrame( pd.read_csv(".\Resources\oscar.csv", index_col='Unnamed: 0') )

# limit to first 6 movies
oscar_df = oscar_df.loc[0:5,:]

# rename oscar df to avoide overlap with imdb data
for name in oscar_df.columns :
    oscar_df = oscar_df.rename( columns={ name : f'Oscar_{name}' })
oscar_df.head()

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination
0,Everything Everywhere All at Once,2022,7,11
1,All Quiet on the Western Front,2022,4,9
2,The Whale,2022,2,3
3,Top Gun: Maverick,2022,1,6
4,Black Panther: Wakanda Forever,2022,1,5


In [3]:
# omdb api preparation
base_url = "http://www.omdbapi.com/?"
params = { "apikey" : ombdapi_key, "type" : "movie" }

# general info to extract from api. column names are key in api.
general_col = ['Year', 'Rated', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', \
               'imdbRating', 'imdbVotes','BoxOffice']

for i in range(len(general_col)) :
    oscar_df[ general_col[i] ] = None

# non omdb rating to fulfill
ratings_col = ['Ratings_IMD', 'Ratings_Rotten_Tomatoes' ,'Ratings_Metacritic']

for i in range(len(ratings_col)) :
    oscar_df[ ratings_col[i] ] = None


In [19]:
# request API

for index in range(len(oscar_df)) :
    
    # export movie by film name in oscar df
    params["t"] = re.sub('\s+', '+', oscar_df.loc[ index , 'Oscar_Film' ] ) 
    movie_data = requests.get( base_url , params ).json()
    
    # extract general info and fulfill the dataframe, by colnames = keys
    for info_col in general_col :
        
        try :
            oscar_df.loc[ index , info_col ] = movie_data[ info_col ]
            
        except KeyError :
            oscar_df.loc[ index , info_col ] = None
            print( f'Film: {params["t"]}. Missing Key: {info_col} ')
    
    # extract other ratings and fill rating columns
    temp_rating = pd.DataFrame( movie_data[ 'Ratings' ] ).set_index("Source")
    
    try :
        oscar_df.loc[ index , 'Ratings_IMD'] = temp_rating.loc[ "Internet Movie Database" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_IMD'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = temp_rating.loc[ "Rotten Tomatoes" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = temp_rating.loc["Metacritic" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = None


In [20]:
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination,Year,Rated,Runtime,Genre,Director,Writer,...,Plot,Language,Country,Awards,imdbRating,imdbVotes,BoxOffice,Ratings_IMD,Ratings_Rotten_Tomatoes,Ratings_Metacritic
0,Everything Everywhere All at Once,2022,7,11,2022,R,139 min,"Action, Adventure, Comedy","Daniel Kwan, Daniel Scheinert","Daniel Kwan, Daniel Scheinert",...,A middle-aged Chinese immigrant is swept up in...,"English, Mandarin, Cantonese",United States,Won 7 Oscars. 372 wins & 357 nominations total,7.8,"445,131","$77,191,785",7.8/10,93%,81/100
1,All Quiet on the Western Front,2022,4,9,2022,R,148 min,"Action, Drama, War",Edward Berger,"Edward Berger, Lesley Paterson, Ian Stokell",...,A young German soldier's terrifying experience...,"German, French","Germany, United States, United Kingdom",Won 4 Oscars. 48 wins & 81 nominations total,7.8,"205,354",N/A,7.8/10,90%,76/100
2,The Whale,2022,2,3,2022,R,117 min,Drama,Darren Aronofsky,Samuel D. Hunter,...,"A reclusive, morbidly obese English teacher at...",English,United States,Won 2 Oscars. 43 wins & 116 nominations total,7.7,"139,972","$17,463,630",7.7/10,64%,60/100
3,Top Gun: Maverick,2022,1,6,2022,PG-13,130 min,"Action, Drama",Joseph Kosinski,"Jim Cash, Jack Epps Jr., Peter Craig",...,"After thirty years, Maverick is still pushing ...",English,United States,Won 1 Oscar. 93 wins & 214 nominations total,8.3,"590,794","$718,732,821",8.3/10,96%,78/100
4,Black Panther: Wakanda Forever,2022,1,5,2022,PG-13,161 min,"Action, Adventure, Drama",Ryan Coogler,"Ryan Coogler, Joe Robert Cole, Stan Lee",...,The people of Wakanda fight to protect their h...,"English, Xhosa, Maya, Spanish, Haitian, French",United States,Won 1 Oscar. 44 wins & 154 nominations total,6.7,"270,203","$453,829,060",6.7/10,84%,67/100
5,Avatar: The Way of Water,2022,1,4,2022,PG-13,192 min,"Action, Adventure, Fantasy",James Cameron,"James Cameron, Rick Jaffa, Amanda Silver",...,Jake Sully lives with his newfound family form...,English,United States,Won 1 Oscar. 63 wins & 129 nominations total,7.7,"388,242","$684,075,767",7.7/10,76%,67/100


'67/100'